In [ ]:
# !pip install dspy-ai
# !pip install -r dspy-requirements-2.txt
!pip install -U dspy-ai

In [1]:
import dspy
import chromadb
from dspy.retrieve.chromadb_rm import ChromadbRM
import json

# Elon Musk's Wikipedia Page as Our Knowledge Source

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

query = "Elon Musk"
raw_documents = WikipediaLoader(query=query).load()

# Chunking

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=100
)
all_splits = text_splitter.split_documents(raw_documents)

# OpenAI ada-002 embedding model

In [5]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = 'openai-api-key'
from langchain_openai import OpenAIEmbeddings
ada_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

Embed our text data with OpenAI ada-002 embedding model in chromadb vector database

In [6]:
from langchain_community.vectorstores import Chroma
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb.utils.embedding_functions as embedding_functions
ADA_CHROMA_COLLECTION_NAME = "dspy-rag-chroma"
ADA_CHROMADB_DIR = "dspy_rag_chroma/"

# Index
ada_vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name=ADA_CHROMA_COLLECTION_NAME,
    embedding=ada_embeddings,
    persist_directory=ADA_CHROMADB_DIR
)

ada_retriever = ada_vectorstore.as_retriever()

ada_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ.get('OPENAI_API_KEY'),
    model_name="text-embedding-ada-002"
)

ada_rm = ChromadbRM(ADA_CHROMA_COLLECTION_NAME, ADA_CHROMADB_DIR, ada_ef, k=3)

# Local embedding - Nomic Embedding

In [11]:
! pip install gpt4all

In [8]:
from langchain_community.embeddings import GPT4AllEmbeddings
nomic_embedding = GPT4AllEmbeddings()

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [9]:
NOMIC_CHROMA_COLLECTION_NAME = "dspy-rag-chroma-nomic"
NOMIC_CHROMADB_DIR = "dspy_rag_chroma_nomic/"

# Index
nomic_vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name=NOMIC_CHROMA_COLLECTION_NAME,
    embedding=nomic_embedding,
    persist_directory=NOMIC_CHROMADB_DIR
)

nomic_retriever = nomic_vectorstore.as_retriever()

nomic_rm = ChromadbRM(NOMIC_CHROMA_COLLECTION_NAME, NOMIC_CHROMADB_DIR, k=3)

In [ ]:
! pip install sentence_transformers

# Build Knowlegde Graph of Elon Musk with Diffbot's Natural Language API

You can now FREELY access all Diffbot APIs (including the Diffbot Natural Language API), which helps construct knowledge graph data and you can further load into any graph database you prefer such as Neo4j, TigerGraph, Kùzu or NebulaGraph) at: https://app.diffbot.com

No more free trials because it's FREE!

In [43]:
# DiffbotGraphTransformer calls Diffbot Natural Language API to extract entities and relationships in the article
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
import os

diffbot_api_key = "Diffbot-token" 

diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

# Diffbot's Natural Language API converts unstructured text data into knowlegde graphs
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

# add knowledge graph data to the neo4j database
graph.add_graph_documents(graph_documents)

In [13]:
# connect to our neo4j database
from langchain_community.graphs import Neo4jGraph

url = "url"
username = "neo4j"
password = "pwd"

graph = Neo4jGraph(url=url, username=username, password=password)

# Vanilla DSPy RAG Pipeline

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class vanilla_dspy_rag(dspy.Module):
    
    # we set num_passages=1 to avoid the same passage being repeatedly retrieved for multiple times
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

# Vanilla DSPy RAG - ada--002 embedding + gpt3.5

In [9]:
# ada--002 embedding model + gpt3.5 language model
lm_gpt35 = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
dspy.settings.configure(lm=lm_gpt35, rm=ada_rm)

question_org_elon_cofounded = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_ada_gpt35 = vanilla_dspy_rag()
vanilla_dspy_rag_q1 = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded)
vanilla_dspy_rag_q1

Prediction(
    context=["In 2004, Musk became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.). He became the company's chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2018, the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequent

In [10]:
# ada--002 embedding model + gpt3.5 language model
question_org_elon_cofounded_1a = "Did Elon Musk co-found SpaceX with other founders?"
vanilla_dspy_rag_q1a = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded_1a)
vanilla_dspy_rag_q1a

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [14]:
# ada--002 embedding model + gpt3.5 language model
question_org_elon_cofounded_1b = "What are the other founders Elon Musk co-founded SpaceX with?"
vanilla_dspy_rag_q1b = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded_1b)
vanilla_dspy_rag_q1b

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [12]:
# ada--002 embedding model + gpt3.5 language model
question_org_elon_cofounded_1c = "Did Elon co-found The Boring Company with other founders?"
vanilla_dspy_rag_q1c = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded_1c)
vanilla_dspy_rag_q1c

Prediction(
    context=['== History ==\n\nMusk announced the idea of the Boring Company in December 2016, and it was officially registered as "TBC – The Boring Company" on January 11, 2017. Musk cited difficulty with Los Angeles traffic, and what he sees as limitations of its two-dimensional transportation network, as his early inspiration for the project. The Boring Company was formed as a SpaceX subsidiary. According to Musk, the company\'s goal is to enhance tunneling speed enough such that establishing a tunnel network is financially feasible. In early 2018, the Boring Company was spun out from SpaceX and into a separate corporate entity. Somewhat less than 10% of equity was given to early employees, and over 90% to Elon Musk. Early employees came from a variety of different backgrounds, including those from SpaceX.\nThe company began designing its own tunnel boring machines, and completed several tests in Hawthorne, California. The Hawthorne test tunnel opened to the public on De

In [13]:
# ada--002 embedding model + gpt3.5 language model
question_org_elon_cofounded_1d = "Is The Boring Company one of the companies Elon co-founded with other founders?"
vanilla_dspy_rag_q1d = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded_1d)
vanilla_dspy_rag_q1d

Prediction(
    context=['== History ==\n\nMusk announced the idea of the Boring Company in December 2016, and it was officially registered as "TBC – The Boring Company" on January 11, 2017. Musk cited difficulty with Los Angeles traffic, and what he sees as limitations of its two-dimensional transportation network, as his early inspiration for the project. The Boring Company was formed as a SpaceX subsidiary. According to Musk, the company\'s goal is to enhance tunneling speed enough such that establishing a tunnel network is financially feasible. In early 2018, the Boring Company was spun out from SpaceX and into a separate corporate entity. Somewhat less than 10% of equity was given to early employees, and over 90% to Elon Musk. Early employees came from a variety of different backgrounds, including those from SpaceX.\nThe company began designing its own tunnel boring machines, and completed several tests in Hawthorne, California. The Hawthorne test tunnel opened to the public on De

In [14]:
# ada--002 embedding model + gpt3.5 language model
question_org_elon_cofounded_1e = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_q1e = vanilla_dspy_rag_ada_gpt35.forward(question=question_org_elon_cofounded_1e)
vanilla_dspy_rag_q1e

Prediction(
    context=["In 2004, Musk became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.). He became the company's chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2018, the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequent

# Vanilla DSPy RAG - ada--002 embedding + Llama3 (as language model)

In [15]:
# ada--002 embedding model + llama3 language model
lm_llama3 = dspy.OllamaLocal(model='llama3', max_tokens=4000)
dspy.settings.configure(lm=lm_llama3, rm=ada_rm)

question_org_elon_cofounded = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_ada_llama3 = vanilla_dspy_rag()
vanilla_dspy_rag_q1 = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded)
vanilla_dspy_rag_q1

Prediction(
    context=["In 2004, Musk became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.). He became the company's chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2018, the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequent

In [16]:
# ada--002 embedding model + llama3 language model
question_org_elon_cofounded_1a = "Did Elon Musk co-found SpaceX with other founders?"
vanilla_dspy_rag_q1a = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded_1a)
vanilla_dspy_rag_q1a

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [17]:
# ada--002 embedding model + llama3 language model
question_org_elon_cofounded_1b = "What are the other founders Elon Musk co-found SpaceX with?"
vanilla_dspy_rag_q1b = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded_1b)
vanilla_dspy_rag_q1b

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [18]:
# ada--002 embedding model + llama3 language model
question_org_elon_cofounded_1c = "Did Elon co-found The Boring Company with other founders?"
vanilla_dspy_rag_q1c = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded_1c)
vanilla_dspy_rag_q1c

Prediction(
    context=['== History ==\n\nMusk announced the idea of the Boring Company in December 2016, and it was officially registered as "TBC – The Boring Company" on January 11, 2017. Musk cited difficulty with Los Angeles traffic, and what he sees as limitations of its two-dimensional transportation network, as his early inspiration for the project. The Boring Company was formed as a SpaceX subsidiary. According to Musk, the company\'s goal is to enhance tunneling speed enough such that establishing a tunnel network is financially feasible. In early 2018, the Boring Company was spun out from SpaceX and into a separate corporate entity. Somewhat less than 10% of equity was given to early employees, and over 90% to Elon Musk. Early employees came from a variety of different backgrounds, including those from SpaceX.\nThe company began designing its own tunnel boring machines, and completed several tests in Hawthorne, California. The Hawthorne test tunnel opened to the public on De

In [19]:
# ada--002 embedding model + llama3 language model
question_org_elon_cofounded_1d = "Is The Boring Company one of the companies Elon co-founded with other founders?"
vanilla_dspy_rag_q1d = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded_1d)
vanilla_dspy_rag_q1d

Prediction(
    context=['== History ==\n\nMusk announced the idea of the Boring Company in December 2016, and it was officially registered as "TBC – The Boring Company" on January 11, 2017. Musk cited difficulty with Los Angeles traffic, and what he sees as limitations of its two-dimensional transportation network, as his early inspiration for the project. The Boring Company was formed as a SpaceX subsidiary. According to Musk, the company\'s goal is to enhance tunneling speed enough such that establishing a tunnel network is financially feasible. In early 2018, the Boring Company was spun out from SpaceX and into a separate corporate entity. Somewhat less than 10% of equity was given to early employees, and over 90% to Elon Musk. Early employees came from a variety of different backgrounds, including those from SpaceX.\nThe company began designing its own tunnel boring machines, and completed several tests in Hawthorne, California. The Hawthorne test tunnel opened to the public on De

In [20]:
# ada--002 embedding model + llama3 language model
question_org_elon_cofounded_1e = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_q1e = vanilla_dspy_rag_ada_llama3.forward(question=question_org_elon_cofounded_1e)
vanilla_dspy_rag_q1e

Prediction(
    context=["In 2004, Musk became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.). He became the company's chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2018, the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequent

In [ ]:
lm_llama3.inspect_history(15)

# Vanilla DSPy RAG - nomic embedding + gpt3.5-instruct

In [21]:
# nomic embedding + gpt3.5-instruct
lm_gpt35 = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
dspy.settings.configure(lm=lm_gpt35, rm=nomic_rm)

question_org_elon_cofounded = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_nomic_gpt35 = vanilla_dspy_rag()
vanilla_dspy_rag_q1 = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded)
vanilla_dspy_rag_q1

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [22]:
# nomic embedding + gpt3.5-instruct
question_org_elon_cofounded_1a = "Did Elon Musk co-found SpaceX with other founders?"
vanilla_dspy_rag_q1a = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded_1a)
vanilla_dspy_rag_q1a

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [18]:
# nomic embedding + gpt3.5-instruct
question_org_elon_cofounded_1b = "What are the other founders Elon Musk co-founded SpaceX with?"
vanilla_dspy_rag_q1b = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded_1b)
vanilla_dspy_rag_q1b

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [24]:
# nomic embeddings + gpt3.5-instruct
question_org_elon_cofounded_1c = "Did Elon co-found The Boring Company with other people?"
vanilla_dspy_rag_q1c = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded_1c)
vanilla_dspy_rag_q1c

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [26]:
# nomic embeddings + gpt3.5-instruct
question_org_elon_cofounded_1d = "Is The Boring Company one of the companies Elon co-founded with other founders?"
vanilla_dspy_rag_q1d = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded_1d)
vanilla_dspy_rag_q1d

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [27]:
# nomic embeddings + gpt3.5-instruct
question_org_elon_cofounded_1e = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_q1e = vanilla_dspy_rag_nomic_gpt35.forward(question=question_org_elon_cofounded_1e)
vanilla_dspy_rag_q1e

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

# Vanilla DSPy RAG - nomic embedding + llama3

In [28]:
dspy.settings.configure(lm=lm_llama3, rm=nomic_rm)

# nomic embedding + llama3
question_org_elon_cofounded = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_nomic_llama3 = vanilla_dspy_rag()
vanilla_dspy_rag_q1 = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded)
vanilla_dspy_rag_q1

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 

In [29]:
# nomic embedding + llama3
question_org_elon_cofounded_1a = "Did Elon Musk co-found SpaceX with other founders?"
vanilla_dspy_rag_q1a = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded_1a)
vanilla_dspy_rag_q1a

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [30]:
# nomic embedding + llama3
question_org_elon_cofounded_1b = "What are the other founders Elon Musk co-found SpaceX with?"
vanilla_dspy_rag_q1b = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded_1b)
vanilla_dspy_rag_q1b

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [31]:
# nomic embedding + llama3
question_org_elon_cofounded_1c = "Did Elon co-found The Boring Company with other people?"
vanilla_dspy_rag_q1c = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded_1c)
vanilla_dspy_rag_q1c

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [32]:
# nomic embedding + llama3
question_org_elon_cofounded_1d = "Is The Boring Company one of the companies Elon co-founded with other founders?"
vanilla_dspy_rag_q1d = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded_1d)
vanilla_dspy_rag_q1d

Prediction(
    context=["== Notable members ==\nMaye Musk (née Haldeman; born 1948), model and dietitian. She has appeared on the cover of several magazines, including a Time magazine health edition, Women's Day, and Vogue; Elon's mother.\nElon Musk (born 1971), entrepreneur and business magnate. Variously CEO, CTO, and/or Chairman of SpaceX, Tesla, X Corp., and Neuralink. He was Time Magazine's 2021 Person of the Year.\nKimbal Musk (born 1972), entrepreneur, philanthropist, and restaurateur. He founded Zip2 in 1998 with his brother Elon and later sold it to Compaq for $307 million. He is the co-founder and chairman of Big Green.\nTosca Musk (born 1974), filmmaker and sister of Elon; she is the co-founder of Passionflix, an OTT entertainment streaming platform and production company.\nLyndon Rive (born 1977), businessman who co-founded SolarCity and served as its CEO until 2017. He is Elon's cousin through his mother Kaye Rive, Maye Musk's twin.\n\n\n== References ==", "== Notable mem

In [33]:
# nomic embedding + llama3
question_org_elon_cofounded_1e = "List out the companies that Elon Musk co-founded with other founders."
vanilla_dspy_rag_q1e = vanilla_dspy_rag_nomic_llama3.forward(question=question_org_elon_cofounded_1e)
vanilla_dspy_rag_q1e

Prediction(
    context=["Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. \nA member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped 